In [ ]:
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from unidecode import unidecode
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
import unicodedata
from difflib import get_close_matches
from funcoes import *
import ipywidgets as widgets
import geopandas as gpd
from shapely import wkt
import folium
import webbrowser
import seaborn as sns
import scipy.stats as st


In [ ]:
pasta = r"C:\Users\Dell\OneDrive\Norm (1)\Norm"

In [ ]:
inventario_macae = carregar_arquivo(
    "INVENTÁRIO_REJEITOS_NORM_MACAE_30_07_25",
    pasta=pasta,
    sheet_name='Base de dados',           # ou o nome da aba, se precisar
    search_subdirs=True,    # vasculha subpastas
    match_mode="fuzzy"      # aceita variações no nome
)

In [ ]:
inventario_tims = carregar_arquivo(
    "Norm/INVENTÁRIO_REJEITOS_NORM_TIMS_30_07_25",
    pasta=pasta,
    sheet_name='Base de dados',           # ou o nome da aba, se precisar
    search_subdirs=True,    # vasculha subpastas
    match_mode="fuzzy"      # aceita variações no nome
)

In [ ]:
#carregar dados da aba Sheet1 e Sheet2
sigep_2003_2024_sheet1 = carregar_arquivo('2003_2024_SIGEP', pasta=pasta, sheet_name='Sheet1', search_subdirs=True, match_mode='fuzzy')



In [ ]:
campos = carregar_arquivo(
    "campos",
    pasta=pasta,
    sheet_name=0,           # ou o nome da aba, se precisar
    search_subdirs=True,    # vasculha subpastas
    match_mode="fuzzy"      # aceita variações no nome
)

# Tratamento dos dados SIGEP

In [ ]:
# #Remover colunas da análise
# sigep_2003_2024_sheet1_filtrado = sigep_2003_2024_sheet1.drop(columns=['Unnamed: 0', 'Estado','Bacia','Nome Poço Operador','Operador','Número do Contrato','Gás Natural (Mm³/dia) Assoc','Gás Natural (Mm³/dia) N Assoc','Volume Gás Royalties (m³/mês)','Tipo Instalação',
#                                                                         'Tempo de Produção (hs por mês)','Corrente',',','Ambiente','Fração de Destilados|Lev|Cort','Fração de Destilados|Méd|Corte',
#                                                                         'Fração de Destilados|Pes|Corte'])

In [ ]:
# #tirar caracter % da col Fração de Destilados|Lev|Vol e Fração de Destilados|Méd|Vol
# colunas = ['Fração de Destilados|Lev|Vol', 'Fração de Destilados|Méd|Vol','Fração de Destilados|Pes|Vol']


# for col in colunas:
#     sigep_2003_2024_sheet1_filtrado[col] = (
#     sigep_2003_2024_sheet1_filtrado[col]
#     .astype(str)
#     .str.replace('%', '', regex=False)
    
#     .str.strip()
#     )

# # agora converte para número com segurança
#     sigep_2003_2024_sheet1_filtrado[col] = pd.to_numeric(
#     sigep_2003_2024_sheet1_filtrado[col],
#     errors='coerce'  # transforma erro em NaN
#     )



In [ ]:
# #renomear colunas modificadas
# sigep_2003_2024_sheet1_filtrado = sigep_2003_2024_sheet1_filtrado.rename(columns={
#     'Fração de Destilados|Lev|Vol': '(%) Fração de Destilados|Lev|Vol',
#     'Fração de Destilados|Méd|Vol': '(%) Fração de Destilados|Méd|Vol',
#     'Fração de Destilados|Pes|Vol': '(%) Fração de Destilados|Pes|Vol'
# })

In [ ]:
# sigep_2003_2024_sheet1_filtrado["Instalação Destino"] = (
#     sigep_2003_2024_sheet1_filtrado["Instalação Destino"]
#     .astype(str)
#     .str.replace(
#         r"(?i)petrobras\s+\d+\s*\(?(P-\d+)?\)?",
#         lambda m: m.group(1) if m.group(1) else "P-" + "".join(filter(str.isdigit, m.group(0))),
#         regex=True
#     )
# )


In [ ]:
# #tirar caracteres malformatados recodificando a coluna 'Instalação Destino'

# sigep_2003_2024_sheet1_filtrado['Instalação Destino'] =  sigep_2003_2024_sheet1_filtrado['Instalação Destino'].astype(str).apply(unidecode)

In [ ]:
# #remover valores nulos do df sigep_2003_2024_sheet1_filtrado
# sigep_2003_2024_sheet1_filtrado['Instalação Destino'] = sigep_2003_2024_sheet1_filtrado['Instalação Destino'].replace(['','NaN', 'nan', 'None'],np.nan)
# sigep_2003_2024_sheet1_filtrado = sigep_2003_2024_sheet1_filtrado.dropna(subset=['Instalação Destino'])

In [ ]:
# #colocar a coluna 'Instalação Destino' como primeira coluna
# sigep_2003_2024_sheet1_filtrado = sigep_2003_2024_sheet1_filtrado[['Instalação Destino']+ [col for col in sigep_2003_2024_sheet1_filtrado.columns if col != 'Instalação Destino']]

In [ ]:
# campos = campos.rename(columns= {'name':'Campo'})

In [ ]:
# sigep_2003_2024_sheet1_filtrado = pd.merge(
#     sigep_2003_2024_sheet1_filtrado,
#     campos[['Campo','geometry']],
#     on='Campo',
#     how = 'left'
# )

In [ ]:
# #remover duplicados 
# sigep_2003_2024_sem_duplicados = sigep_2003_2024_sheet1_filtrado.drop_duplicates(subset=['Óleo (bbl/dia)'])
# sigep_2003_2024_sem_duplicados['Ano']= pd.to_datetime(sigep_2003_2024_sem_duplicados['Período'], format="%Y/%m").dt.year

In [ ]:
# # 1. Converter coluna Período para datetime mensal
# sigep_2003_2024_sem_duplicados["Período"] = pd.to_datetime(
#     sigep_2003_2024_sem_duplicados["Período"].astype(str),
#     format="%Y/%m",   # espera valores tipo 2003/01
#     errors="coerce"
# )

# # 2. Criar colunas auxiliares
# sigep_2003_2024_sem_duplicados["Ano"] = sigep_2003_2024_sem_duplicados["Período"].dt.year
# sigep_2003_2024_sem_duplicados["Mes"] = sigep_2003_2024_sem_duplicados["Período"].dt.month
# sigep_2003_2024_sem_duplicados["AnoMes"] = sigep_2003_2024_sem_duplicados["Período"].dt.to_period("M").astype(str)
# sigep_2003_2024_sem_duplicados["Dias_mes"] = sigep_2003_2024_sem_duplicados["Período"].dt.days_in_month

# # 3. Identificar colunas de taxa (bbl/dia ou Mm³/dia)
# colunas_taxa = [c for c in sigep_2003_2024_sem_duplicados.columns if "(bbl/dia)" in c or "(Mm³/dia)" in c]

# # 4. Converter taxas diárias em volumes mensais
# for col in colunas_taxa:
#     novo_nome = col.replace("(bbl/dia)", "(bbl/mês)").replace("(Mm³/dia)", "(Mm³/mês)")
#     sigep_2003_2024_sem_duplicados[novo_nome] = (
#         sigep_2003_2024_sem_duplicados[col] * sigep_2003_2024_sem_duplicados["Dias_mes"]
#     )

# # 5. Produção mensal consolidada
# producao_mensal = (
#     sigep_2003_2024_sem_duplicados
#     .groupby(["AnoMes", "Instalação Destino"])[[c for c in sigep_2003_2024_sem_duplicados.columns 
#                                                if "(bbl/mês)" in c or "(Mm³/mês)" in c]]
#     .sum()
#     .reset_index()
# )

# # 6. Identificar colunas de composição (% e Grau API)
# colunas_comp = [c for c in sigep_2003_2024_sem_duplicados.columns if "% em Volumes" in c or "Grau API" in c or '(%) Fração' in c]

# # 7. Função para média ponderada (usando petróleo como peso)
# def media_ponderada(grupo):
#     peso = grupo["Petróleo (bbl/mês)"]
#     if peso.sum() == 0:  # evita divisão por zero
#         return grupo[colunas_comp].mean()
#     return (grupo[colunas_comp].multiply(peso, axis=0).sum() / peso.sum())

# # 8. Calcular composição mensal ponderada
# comp_mensal = (
#     sigep_2003_2024_sem_duplicados
#     .groupby(["AnoMes", "Instalação Destino"], group_keys=False)
#     .apply(media_ponderada)
#     .reset_index()
# )

# # 9. Juntar Produção + Composição
# resultado_final = producao_mensal.merge(comp_mensal, on=["AnoMes", "Instalação Destino"])

# # 10. Exportar CSV
# resultado_final.to_csv("resultado_mensal.csv", index=False, encoding="utf-8-sig")

# print("✅ Processamento concluído. Resultado disponível no DataFrame 'resultado_final'")
# print(resultado_final.head())




# Tratamento de dados INVENTARIO

In [ ]:
inventario_tims.columns = inventario_tims.iloc[0]

inventario_tims = inventario_tims[1:].reset_index(drop=True)

In [ ]:
#remover colunas vazias

inventario_macae = inventario_macae.dropna(axis=1, how='all')
inventario_tims = inventario_tims.dropna(axis=1, how='all')

In [ ]:
#10 locais que mais geram 

top_locais = inventario_macae['LOCAL DA GERAÇÃO'].value_counts().head(10)

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(x=top_locais.values, y=top_locais.index, color='steelblue')
plt.title('Quantidade de volumes por Local de Geração')
plt.ylabel('Local de Geração')
plt.xlabel('Quantidade de volumes')

plt.show()

In [ ]:
top_quantidades = (inventario_macae.groupby('LOCAL DA GERAÇÃO')['MASSA DO VOLUME (kg)'].sum().sort_values(ascending=False).head(10))

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(x=top_quantidades.values, y=top_quantidades.index, color='darkorange')
plt.title('Massa de volume por Local de Geração')
plt.ylabel('Local de Geração')
plt.xlabel('Massa de volume (kg)')

In [ ]:
inventario_macae['TIPO DE RESÍDUO'].unique()

In [ ]:
inventario_tims[' TIPO DE RESÍDUO'].unique()

In [ ]:
map_residuos = {
    'BORRA CAT.II':'BORRA CAT II',
    'BORRA CAT.I':'BORRA CAT I'
}

inventario_macae['TIPO DE RESÍDUO'] = (inventario_macae['TIPO DE RESÍDUO'].replace(map_residuos).str.upper().str.strip())

In [ ]:
residuos_por_galpao = (inventario_macae.groupby(['DEPÓSITO', 'TIPO DE RESÍDUO'])['MASSA DO VOLUME (kg)'].sum().reset_index())

In [ ]:
pivot_residuos = residuos_por_galpao.pivot_table(
    index="DEPÓSITO",
    columns="TIPO DE RESÍDUO",
    values="MASSA DO VOLUME (kg)",
    aggfunc="sum",
    fill_value=0
)

In [ ]:
pivot_pct = pivot_residuos.div(pivot_residuos.sum(axis=1), axis=0) * 100


In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(
    pivot_pct,
    cmap="Blues",
    annot=True,
    fmt=".1f",
    cbar_kws={'label': '% do resíduo no galpão'},
    linewidths=0.5,
    linecolor="gray"
)
plt.title("Distribuição percentual de tipos de resíduos por galpão")
plt.xlabel("Tipo de Resíduo")
plt.ylabel("Galpão / Depósito")
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(
    pivot_residuos,
    cmap="OrRd",
    annot=True,
    fmt=".0f",
    cbar_kws={'label': 'Massa (kg)'},
    linewidths=0.5,
    linecolor="gray"
)
plt.title("Distribuição absoluta de tipos de resíduos por galpão")
plt.xlabel("Tipo de Resíduo")
plt.ylabel("Galpão / Depósito")
plt.tight_layout()
plt.show()


In [ ]:
inventario_macae['TIPO DE RESÍDUO'].unique()

In [ ]:
# Criar AnoMes no formato YYYY-MM
inventario_macae["AnoMes"] = inventario_macae["ANO DE GERAÇÃO (DATA SIGRE)"].dt.to_period("M").astype(str)

# 1) Soma mensal por depósito + tipo de resíduo
soma_mensal = (
    inventario_macae
    .groupby(["AnoMes","LOCAL DA GERAÇÃO","TIPO DE RESÍDUO"], as_index=False)["MASSA DO VOLUME (kg)"]
    .sum()
    .reset_index(drop=True)   # garante DataFrame
    .sort_values(["LOCAL DA GERAÇÃO","TIPO DE RESÍDUO","AnoMes"])   
)

# 2) Acumulado correto por depósito + tipo
soma_mensal["Acumulado (kg)"] = (
    soma_mensal.groupby(["LOCAL DA GERAÇÃO","TIPO DE RESÍDUO"])["MASSA DO VOLUME (kg)"].cumsum()
)

# 3) Pivot para formato largo
acumulado_residuo = (
    soma_mensal
    .pivot_table(
        index=["AnoMes","LOCAL DA GERAÇÃO"],
        columns="TIPO DE RESÍDUO",
        values="Acumulado (kg)",
        aggfunc="last",
        fill_value=0
    )
    .reset_index()
)

# 4) Renomear colunas para deixar claro
acumulado_residuo = acumulado_residuo.rename(
    columns=lambda x: f"Acumulado {x}" if x not in ["AnoMes","LOCAL DA GERAÇÃO"] else x
)

print(acumulado_residuo.head())


In [ ]:


deposito = "FPSO-CST"
dados = acumulado_residuo[acumulado_residuo["LOCAL DA GERAÇÃO"] == deposito]

# Transformar dados de wide → long
dados_long = dados.melt(
    id_vars=["AnoMes", "LOCAL DA GERAÇÃO"],
    value_vars=[c for c in dados.columns if c.startswith("Acumulado")],
    var_name="Tipo de Resíduo",
    value_name="Acumulado (kg)"
)

# Deixar o nome mais limpo
dados_long["Tipo de Resíduo"] = dados_long["Tipo de Resíduo"].str.replace("Acumulado ", "")

# Gráfico interativo
fig = px.line(
    dados_long,
    x="AnoMes",
    y="Acumulado (kg)",
    color="Tipo de Resíduo",
    title=f"Evolução acumulada de resíduos - {deposito}"
)

fig.update_traces(mode='markers+lines')
# Ajustes visuais
fig.update_layout(
    xaxis=dict(tickangle=-45),
    legend=dict(title="Clique para ligar/desligar resíduos")
)

fig.show()


# Unir SIGEP e INVENTARIO MACAÉ

In [ ]:
#remover colunas que n]ao serão utilizadas

inventario_macae_filtrado = inventario_macae[['DEPÓSITO', 'TIPO DE RESÍDUO', 'AnoMes', 'MASSA DO VOLUME (kg)','LOCAL DA GERAÇÃO','TAXA DE DOSE (µSv/h) DEPÓSITO']]
inventario_macae_filtrado = inventario_macae_filtrado.rename(columns={'LOCAL DA GERAÇÃO':'Instalação Destino'})

In [ ]:
# unidos = resultado_final.merge(inventario_macae_filtrado,
#                                on=['Instalação Destino','AnoMes'],
#                                how='left',
#                                suffixes=('', '_inventario')
# )

# Análise por Instalação destino

In [ ]:
inventario_macae['Ano'] = pd.to_datetime(inventario_macae['AnoMes'], errors = 'coerce').dt.year.astype('Int64')


In [ ]:
print(inventario_macae[inventario_macae['LOCAL DA GERAÇÃO']== 'FPSO-CST']['Ano'].unique())

In [ ]:
zeros = inventario_macae_filtrado[inventario_macae_filtrado["MASSA DO VOLUME (kg)"] == 0]
print("Total de registros iguais a zero:", len(zeros))



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
inventario_macae['Ano'] = pd.to_datetime(inventario_macae['AnoMes'], errors = 'coerce').dt.year.astype('Int64')

# anos só da FPSO-CST
anos = sorted(
    inventario_macae[inventario_macae['LOCAL DA GERAÇÃO'] == 'FPSO-CST']['Ano']
    .dropna()
    .unique()
)

plt.figure(figsize=(12, 8))
for ano in anos:
    data = inventario_macae[
        (inventario_macae['Ano'] == ano) &
        (inventario_macae['LOCAL DA GERAÇÃO'] == 'FPSO-CST') &
        (inventario_macae['MASSA DO VOLUME (kg)'] > 0)
    ]['MASSA DO VOLUME (kg)'].dropna()

    if len(data) > 5:
        alpha, loc, beta = st.gamma.fit(data, floc=0)
        x = np.linspace(min(data), max(data), 200)
        pdf = st.gamma.pdf(x, alpha, loc=loc, scale=beta)

        plt.hist(data, bins=30, density=True, alpha=0.5)
        plt.plot(x, pdf, label=f"Gama {ano} (α={alpha:.1f}, β={beta:.1f})")

plt.title('Distribuição de resíduos por ano (Gama ajustada) - FPSO-CST')
plt.xlabel('Massa do Volume (kg)')
plt.ylabel('Densidade')
plt.legend()
plt.grid(True)
plt.show()




In [ ]:
popt_alpha, popt_scale, popt_loc, params_df = regressao_parametros_gamma(
    inventario_macae,
    unidade='P-48',
    parametro='MASSA DO VOLUME (kg)',
    regressao='linear',   # pode trocar por 'polinomial' ou 'exponencial'
    floc=True,
    plotar=True           # exibe os gráficos lado a lado
)
